In [ ]:
from src import Color
from src.material.material.material import create_phong_material
from src import World
from src import Sphere, Plane
from src import PointLight, AmbientLight
from src import ipynb_display_image
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene

# 4 spheres in a row with different IOR (index of refraction) values
1. oil with IOR=1.2
2. glass with IOR=1.33
3. crystal with IOR=1.5
4. diamond with IOR=2.4
#### All spheres have the same transparency (0.95) and shininess (64). The ground plane is matte gray to better see the refraction effects on the HDR skybox background.

In [ ]:
# colors
brown = Color.custom_rgb(150, 75, 0)
red   = Color.custom_rgb(255, 0, 0)
white = Color.custom_rgb(245, 245, 245)
green = Color.custom_rgb(30, 180, 120)
gray  = Color.custom_rgb(200, 200, 200)

# --- material (vary IOR only) ---
glass_spec = Color.custom_rgb(255, 255, 255)


ior_oil     = create_phong_material("ior_air",     base_color=red, spec_color=glass_spec,
                                    shininess=64, ior=1.2, reflectivity=0.0, transparency=0.95)
ior_glass   = create_phong_material("ior_glass",   base_color=red, spec_color=glass_spec,
                                    shininess=64, ior=1.33, reflectivity=0.0, transparency=0.95)
ior_crystal = create_phong_material("ior_crystal", base_color=red, spec_color=glass_spec,
                                    shininess=64, ior=1.5, reflectivity=0.0, transparency=0.95)
ior_diamond = create_phong_material("ior_diamond", base_color=red, spec_color=glass_spec,
                                    shininess=64, ior=2.4, reflectivity=0.0, transparency=0.95)


matte_ground = create_phong_material("matte_ground", base_color=gray,
                                     spec_color=Color.custom_rgb(80, 80, 80),
                                     shininess=8, reflectivity=0.0)


sphere1 = Sphere(center=Vertex(-3.5, 0.2, -5), radius=1.0, material=ior_oil)
sphere2 = Sphere(center=Vertex(-1.1, 0.2, -5), radius=1.0, material=ior_glass)
sphere3 = Sphere(center=Vertex( 1.1, 0.2, -5), radius=1.0, material=ior_crystal)
sphere4 = Sphere(center=Vertex( 3.5, 0.2, -5), radius=1.0, material=ior_diamond)

# ground plane
plane = Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0), material=matte_ground)

# --- world ---
world = World()
for obj in (sphere1, sphere2, sphere3, sphere4, plane):
    world.add(obj)

# --- lights ---
point_light_1 = PointLight(position=Vertex(5, 6, -1), intensity=1600.0, falloff=0.01)
point_light_2 = PointLight(position=Vertex(0, 5, -5), intensity=1200.0, falloff=0.01)
ambient_light = AmbientLight(intensity=0.2)
lights = [point_light_1, point_light_2, ambient_light]

# --- camera / scene ---
camera = Camera(
    fov=65,
    resolution=Resolution.R360p,
    origin=Vertex(0, 0, 0),
    direction=Vector(0, 0, -1),
)

scene = Scene(
    camera=camera,
    world=world,
    lights=lights,
    skybox_path="./skybox/shanghai_4k.hdr"
)

scene.validate()
scene.render_phong(
    samples_per_pixel=2,
    max_depth=6,
    image_png_path="./images/ior_test.png"
)

ipynb_display_image("./images/ior_test.png")


In [ ]:
scene.render_multithreaded(
    samples_per_pixel=2,
    max_depth=6,
    image_png_path="./images/ior_test_multithreaded.png"
)
ipynb_display_image("./images/ior_test_multithreaded.png")